In [ ]:
from astroquery.sdss import SDSS

from astropy import coordinates as coords

import astropy.units as u

import matplotlib.pyplot as plt

import numpy as np

from PIL import Image

import requests

from io import BytesIO

from openpyxl import load_workbook


# === Step 1: Set your galaxy coordinates (RA and DEC) ===



# Example galaxy: NGC 7609 — has strong H-alpha line

# RA = 349.87521  # Right Ascension (in degrees)

# DEC = 9.50822   # Declination (in degrees)


wb = load_workbook('SDSS_100_Galaxies.xlsx')
sheet = wb.active

for i in range(3, 104):
    RA = sheet[f'B{i}'].value
    DEC = sheet[f'C{i}'].value


    SEARCH_RADIUS = 0.005



    # === Step 2: Tell the computer where to look in the sky ===

    position = coords.SkyCoord(ra=RA, dec=DEC, unit='deg')

    print("Searching for galaxy images near this sky position...")



    # === Step 3: Ask SDSS for any galaxy images at this position ===

    try:

        results = SDSS.query_region(position, radius=SEARCH_RADIUS * u.deg, photoobj_fields=['ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'objid', 'type', 'petroMag_r', 'petroR50_r', 'petroR90_r'])

    except Exception as e:

        print("That coordinate didn't work!")


    # If nothing is found, stop

    if results is None or len(results) == 0:

        print("No galaxy images found here.")

        exit()



    print(f"Found {len(results)} objects with imaging!")


    if results is None or len(results) == 0 or 'ra' not in results.colnames:
        print(f"Row {i}: No usable data found, skipping.")
        continue

    # === Step 4: Get the first available galaxy (type = 3 means galaxy) ===
    if 'type' in results.colnames:
        galaxy_objects = results[results['type'] == 3]  # Filter for galaxies only

        if len(galaxy_objects) == 0:

            print("No galaxies found in the results, using first object instead.")

            first = results[0]

        else:

            first = galaxy_objects[0]
    
    else:
        print(f"'type' column not found in results, using first object instead.")
        first = results[0]



    print(f"Selected object: RA={first['ra']:.6f}, DEC={first['dec']:.6f}")



    # === Step 5: Download the composite image ===

    print("Downloading color composite image...")



    # Create URL for SDSS color image

    ra_center = first['ra']

    dec_center = first['dec']

    scale = 0.2  # arcsec per pixel

    width = 512  # in pixels

    height = 512 # in pixels



    img_url = f"https://skyserver.sdss.org/dr17/SkyServerWS/ImgCutout/getjpeg?ra={ra_center}&dec={dec_center}&scale={scale}&width={width}&height={height}"



    # === Step 6: Show the galaxy! ===

    try:

        response = requests.get(img_url)

        if response.status_code == 200:

            img = Image.open(BytesIO(response.content))

            plt.figure(figsize=(8, 8))

            plt.imshow(img)

            plt.title(f'SDSS Color Composite\nRA={first["ra"]:.4f}, DEC={first["dec"]:.4f}')

            plt.axis('off')

            plt.show()

        else:

            print("Color image download failed")

    except Exception as e:

        print(f"Category unknown: {e}") 


    R50 = first['petroR50_r']  # Petrosian radius at 50% light in r-band
    R90 = first['petroR90_r']  # Petrosian radius at 90% light in r-band
    v = R90 / R50  # Calculate the ratio of R90 to R50
    sheet[f'J{i}'] = v  # Store the ratio in column J of the Excel sheet
    try:
        if v < 2.2:
            classification = "Late-type spiral or irregular galaxy (disk-dominated)"
        elif 2.2 <= v < 2.6:
            classification = "Intermediate spiral galaxy (disk + bulge)"
        else:
            classification = "Elliptical or bulge-dominated spiral galaxy"

        print(f"Galaxy type: {classification}")

        sheet[f'K{i}'] = classification  # Store the classification in column K of the Excel sheet

    except Exception as e:  
        print(f"Category unknown: {e}")
        sheet[f'K{i}'] = "Classification Error"

wb.save('SDSS_100_Galaxies_output.xlsx')

print("=== Done! ===")

In [ ]:
from astroquery.sdss import SDSS

from astroquery.simbad import Simbad

from astropy.coordinates import SkyCoord

import astropy.units as u

import numpy as np

import pandas as pd



input_file = 'SDSS_100_Galaxies_output.xlsx'
df = pd.read_excel(input_file, header=1)  # Read the Excel file, skipping the first row


RA_col = 'ra'
DEC_col = 'dec'
z_col = 'redshift'  # Assuming the redshift column is named 'redshift'

df['Distance_km'] =np.nan  # Initialize redshift column
df['velocity_(km/s)'] = np.nan  # Initialize velocity column

for index, row in df.iterrows():
    ra = row[RA_col]
    dec = row[DEC_col]
    z = row[z_col]

    SEARCH_RADIUS = 0.001
      # degrees
    try:
      position = SkyCoord(ra=ra, dec=dec, unit='deg')
      results = SDSS.query_region(position, radius=SEARCH_RADIUS * u.deg, spectro=True, specobj_fields=['plate', 'mjd', 'fiberID', 'z'])

      if results is None or len(results) == 0:
        print("No galaxy images found here.")
      exit()



      c = 299792.458  # Speed of light in km/s
      if z < 0.1:
        velocity = abs(z * c)
      else:
        velocity = abs(c * ((1 + z)**2 - 1) / ((1 + z)**2 + 1))

      df.at[index, 'velocity_(km/s)'] = velocity  # Store velocity in the DataFrame

      H0 = 72 # km/s/Mpc
      H = H0 * (1 + z)  # Hubble parameter at redshift z
      D = ((velocity / H)*u.Mpc).to(u.km) # in Mpc
      Distance=(f"{D:.2e}".replace('e+', ' × 10^'))
      df.at[index, 'Distance_km'] = Distance  # Store distance in the DataFrame

    except Exception as e:
      print(f" ERROR: {e}")

output_file = 'SDSS_100_Galaxies_redshift.xlsx'
df.to_excel(output_file, index=False)
print(f"Redshift and velocity data saved to {output_file}")